In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
#Define function to extract data from file, Load training data and convert into Numpy array
def get_data(file_name):
    df_input=pd.read_csv(file_name,usecols =['T1','T2','T_out','Press_mm_hg','Windspeed'])
    #'RH_out','T3', 'T4','Visibility','Tdewpoint'
    df_target=pd.read_csv(file_name,usecols =['Appliances'])
    print(df_input.head())       #Print first few columns of input data
    print(df_target.head())      #Print first few columns of target data
    #print(df_input.info())
    #print(df_target.info())
    X_data = df_input.as_matrix()
    y_data = df_target.as_matrix()
    X_data.astype('float32', copy=False)
    y_data.astype('float32', copy=False)
    
    X_train, X_valid, y_train, y_valid = train_test_split( X_data, y_data,test_size=0.20)  #function to split data set
    
    
    return X_train,y_train,X_valid,y_valid


In [3]:
#Get training data
file_name = './Appliances_energy_prediction_data_set_UCI.csv'
X_train,y_train,X_valid,y_valid = get_data(file_name)
print('Shape of training input data set',X_train.shape)
print('Shape of training target data set',y_train.shape)
print('Number of training samples:',len(X_train))
print('Shape of validation input data set',X_valid.shape)
print('Shape of validation target data set',y_valid.shape)

      T1    T2  T_out  Press_mm_hg  Windspeed
0  19.89  19.2   6.60        733.5   7.000000
1  19.89  19.2   6.48        733.6   6.666667
2  19.89  19.2   6.37        733.7   6.333333
3  19.89  19.2   6.25        733.8   6.000000
4  19.89  19.2   6.13        733.9   5.666667
   Appliances
0          60
1          60
2          50
3          50
4          60
Shape of training input data set (15788, 5)
Shape of training target data set (15788, 1)
Number of training samples: 15788
Shape of validation input data set (3947, 5)
Shape of validation target data set (3947, 1)


In [4]:
#Function to normalize input data along features
def normalize_data(raw_data):
    import tensorflow as tf
    return tf.nn.l2_normalize(raw_data,dim=0)

In [5]:
#Training parametres
epochs = 10     #Number of times to peform training
display_epoch = 2  
learning_rate = 0.001  #The rate at which weights are updated in the neural network
mini_batch_size = 100  #The size to which the training data is split up

In [6]:
#Network Parameteres
L1 = 10 # Number of neurons in Layer 1

In [7]:
#Placeholders for input and output
Xfeatures = 5  #Number of features in input data
Yfeatures =1    #Number of features in target data
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))#Place holder for input data
#Output
Y = tf.placeholder(tf.float32,shape=(None,Yfeatures))   #Place holder for target data

In [8]:
#Builda single layer model using core TensorFlow operations
def FFNN_using_core(Xinput):          #Function containing FFNN model
    #Layer1 
    W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1])) # Weights
    b_fc1 = tf.Variable(tf.random_uniform([L1]))        #Biases
    matmul_fc1=tf.matmul(Xinput, W_fc1) + b_fc1                #Matrix multiplication and addition
    h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation function

    #Output layer 
    W_fO= tf.Variable(tf.random_uniform([L1,Yfeatures])) # Weights
    b_fO = tf.Variable(tf.random_uniform([Yfeatures]))    #Biases
    matmul_fcO=tf.matmul(h_fc1, W_fO) + b_fO    #Matrix multiplication and additon
    output_layer = matmul_fcO  #linear activation
    return output_layer

prediction1 = FFNN_using_core(normalize_data(X))       #Call function containing for model
#prediction = FFNN_using_core(X)       #Call function
MAE1= tf.metrics.mean_absolute_error(Y,prediction1)   #Function to calculate mean absolute error 

#Loss function
loss1 = tf.reduce_mean(tf.square(Y-prediction1))

#Specify optimizer
train_step1 = tf.train.AdamOptimizer(learning_rate).minimize(loss1)

In [9]:
#Build single layer model using TensorFlow layers
def FFNN_using_layer(Xinput):          #Function containing FFNN model
    #Layer1 
    FC1 = tf.layers.dense(Xinput,L1,activation=tf.nn.relu)   #Add a feed forward layer

    #Output layer 
    output_layer = tf.layers.dense(FC1,Yfeatures)
    return output_layer

prediction2 = FFNN_using_layer(normalize_data(X))       #Call function
#prediction = FFNN_using_layer(X)       #Call function
MAE2= tf.metrics.mean_absolute_error(Y,prediction2)
#Specify Loss function
loss2 = tf.losses.mean_squared_error (Y,prediction2)
#Specify optimizer
train_step2 = tf.train.AdamOptimizer(learning_rate).minimize(loss2)

In [10]:
#Training session
init = tf.global_variables_initializer()
init_local = tf.local_variables_initializer() 

#Operation to create mini-batches of shuffled data

#Select which model to use
loss = loss1
train_step = train_step1
MAE = MAE1

with tf.Session() as sess:
    sess.run([init,init_local])
    for i in range(epochs):
        for j in range(int(len(X_train)/mini_batch_size)):
            X1,y1 = shuffle(X_train,y_train, n_samples=mini_batch_size)
            sess.run(train_step,feed_dict={X:X1,Y:y1})
        #sess.run(train_step,feed_dict={X:X_train,Y:y_train})
        if i%display_epoch ==0:
            training_loss = sess.run(loss,feed_dict={X:X_train,Y:y_train})
            validation_loss = sess.run(loss,feed_dict={X:X_valid,Y:y_valid})
            print("Training loss (Mean squre error) at epoch {}:{}".format(i+1,training_loss))
            print("Validation loss (Mean squre error) at epoch {}:{}".format(i+1,training_loss))
    _,mae = sess.run(MAE,feed_dict={X:X_train,Y:y_train})
    print("Mean Absolute Error:",mae)

Training loss (Mean squre error) at epoch 1:18851.4140625
Validation loss (Mean squre error) at epoch 1:18851.4140625
Training loss (Mean squre error) at epoch 3:17702.17578125
Validation loss (Mean squre error) at epoch 3:17702.17578125
Training loss (Mean squre error) at epoch 5:16129.3720703125
Validation loss (Mean squre error) at epoch 5:16129.3720703125
Training loss (Mean squre error) at epoch 7:14479.2724609375
Validation loss (Mean squre error) at epoch 7:14479.2724609375
Training loss (Mean squre error) at epoch 9:13107.0986328125
Validation loss (Mean squre error) at epoch 9:13107.0986328125
Mean Absolute Error: 51.8416


In [12]:
#Inference session
with tf.Session() as sess:
    sess.run([init,init_local])
    index = random.randint(0, len(X_valid))  #Find random input vector from dataset 
    prediction,mae = sess.run([prediction1,MAE],feed_dict={X:X_valid[index].reshape(1,5),Y:y_valid[index].reshape(1,1)})
    print("Predicted load for input vector {}:{}".format(X_valid[index,:],*prediction))
    print("Actual value is {} and Mean Absolute Error is {}".format(y_valid[index],mae[1]))

Predicted load for input vector [  21.29         18.6           5.77        753.9666667     2.66666667]:[ 13.39552879]
Actual value is [60] and Mean Absolute Error is 46.604469299316406


In [13]:
#reset earlier graphs
tf.reset_default_graph() 

#Building and training a single layer model using Keras (Available within TensorFlow)
model = tf.contrib.keras.models.Sequential() 
#Input Layer
model.add(tf.contrib.keras.layers.InputLayer(input_shape=(Xfeatures,),name='InputLayer'))
model.add(tf.contrib.keras.layers.BatchNormalization(axis=1))  #Normalizing values
#model.add(tf.contrib.keras.layers.Lambda(normalize_data ,name='Normalizing'))  #Normalizing values
#Layer1 
model.add(tf.contrib.keras.layers.Dense(units=L1,activation='relu',name='FeedForward1'))  #Add a feed forward layer
#Layer2 
model.add(tf.contrib.keras.layers.Dense(units=L1,activation='relu',name='FeedForward2'))  #Add a feed forward layer
#Output layer 
model.add(tf.contrib.keras.layers.Dense(units=Yfeatures,name='OutputLayer'))

#Specify los function and optimizer
model.compile(loss='mse',optimizer='adam',metrics=['mae'])

#Summarize model
model.summary()
#Callbacks
tb_callback = tf.contrib.keras.callbacks.TensorBoard(log_dir='./logs',histogram_freq=2,write_images=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      (None, 5)                 0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5)                 20        
_________________________________________________________________
FeedForward1 (Dense)         (None, 10)                60        
_________________________________________________________________
FeedForward2 (Dense)         (None, 10)                110       
_________________________________________________________________
OutputLayer (Dense)          (None, 1)                 11        
Total params: 201
Trainable params: 191
Non-trainable params: 10
_________________________________________________________________


In [26]:
# Start Training model
model.fit(X_train,y_train,batch_size=mini_batch_size,epochs=epochs,validation_data=(X_valid,y_valid),shuffle =True,callbacks=[tb_callback],verbose=1)
#,callbacks=[tb_callback],
#,validation_split = 0.2
model.save('demo.h5')
print("Model Saved")

Train on 15788 samples, validate on 3947 samples
Epoch 1/10
15788/15788 [==============================] - 1s - loss: 9873.6254 - mean_absolute_error: 57.9533 - val_loss: 10646.9941 - val_mean_absolute_error: 58.9535
Epoch 2/10
15788/15788 [==============================] - 0s - loss: 9874.9814 - mean_absolute_error: 57.3347 - val_loss: 10635.7632 - val_mean_absolute_error: 60.6774
Epoch 3/10
15788/15788 [==============================] - 1s - loss: 9869.3934 - mean_absolute_error: 57.7132 - val_loss: 10634.7616 - val_mean_absolute_error: 59.0824
Epoch 4/10
15788/15788 [==============================] - 0s - loss: 9859.1671 - mean_absolute_error: 57.3788 - val_loss: 10626.9434 - val_mean_absolute_error: 60.7168
Epoch 5/10
15788/15788 [==============================] - 1s - loss: 9869.5264 - mean_absolute_error: 57.9836 - val_loss: 10621.7771 - val_mean_absolute_error: 59.1147
Epoch 6/10
15788/15788 [==============================] - 0s - loss: 9856.1340 - mean_absolute_error: 57.3337 -

In [27]:
#Inference session
model.predict(X_valid,batch_size=10)

array([[ 105.66149902],
       [ 135.91659546],
       [ 132.8243103 ],
       ..., 
       [  71.93901825],
       [  71.56589508],
       [  95.65309143]], dtype=float32)

In [28]:
#Inference session
index = random.randint(0, len(X_valid))  #Find random input vector from dataset 
prediction = model.predict(X_valid,batch_size=1) #Find predicted value on random data set
print("Predicted load for input vector {}:{}".format(X_valid[index],*prediction))
print("Actual value is {} and  Absolute Error is {}".format(y_valid[index],abs(prediction[0]-y_valid[index])))

Predicted load for input vector [  22.2         21.6          8.77       751.1333333    7.       ]:[ 105.66149902]
Actual value is [400] and  Absolute Error is [ 294.33850098]


In [ ]:
#Tensorboard
#tensorboard --logdir=C:\TensorFlow_tutorial\logs